In [2]:

println("Number of threads: ", Threads.nthreads())


Number of threads: 7


In [4]:
using PyCall
using DataFrames

# Import pandas via PyCall
pd = pyimport("pandas")

# Load the pickle file
df = pd.read_pickle("/content/ml_dataset_50000_float.pkl")

# Define a function to convert NumPy arrays with complex numbers to Julia arrays
function numpy_to_julia(arr)
    # Convert each element in the 2D array from PyObject encapsulating complex numbers to Julia's ComplexF64
    # Assuming arr is a 2D Julia Array with ComplexF64 elements
    return [complex(real(arr[i, j]), imag(arr[i, j])) for i in 1:size(arr, 1), j in 1:size(arr, 2)]
end

# Convert the DataFrame to a Julia DataFrame
# Assuming the complex array column is named "Target State" and the boolean column is named "Local"
julia_df = DataFrame(
    State = [numpy_to_julia(convert(Array{ComplexF64}, df["Target State"][i + 1])) for i in 0:length(df["Target State"]) - 1],
    SDPValue = [df["SDP Value"][i + 1] for i in 0:length(df["SDP Value"]) - 1]  # Convert PyObject to Julia Bool
)

# Now julia_df contains the data from your pickle file, converted to a suitable format for Julia
first(julia_df, 10)


Row,State,SDPValue
,Array…,Float64
1,ComplexF64[0.114158+6.52071e-18im 1.42405+0.376435im 1.61559-0.154313im 1.45879-1.95898im; 1.42405-0.376435im 0.270139-3.60968e-18im 0.0466473-0.0597235im 0.292622-0.509763im; 1.61559+0.154313im 0.0466473+0.0597235im 0.168194+9.60731e-18im 0.948293+2.3808im; 1.45879+1.95898im 0.292622+0.509763im 0.948293-2.3808im 0.447509-1.25184e-17im],0.05073
2,ComplexF64[0.0424817-1.12918e-18im 1.30677-1.10884im 0.365089-0.292983im 0.344913+0.00182559im; 1.30677+1.10884im 0.0720195-7.23822e-18im 0.0608774+0.0128727im 0.14322-0.0354642im; 0.365089+0.292983im 0.0608774-0.0128727im 0.14927-7.63868e-18im 0.0428507+0.0610667im; 0.344913-0.00182559im 0.14322+0.0354642im 0.0428507-0.0610667im 0.736229+1.60061e-17im],0.0933097
3,ComplexF64[0.28904+5.61945e-18im 0.198964-0.237838im 0.23586-0.234188im 0.370936-0.919555im; 0.198964+0.237838im 0.391603+2.90452e-19im 0.19401+0.00796868im 0.493137+0.503926im; 0.23586+0.234188im 0.19401-0.00796868im 0.257568-2.08347e-18im 0.0449028-0.0769871im; 0.370936+0.919555im 0.493137-0.503926im 0.0449028+0.0769871im 0.0617895-3.82642e-18im],0.162827
4,ComplexF64[0.201138-9.26598e-18im 0.100415+0.0368004im 0.0269368-0.0124297im 0.0618175+0.00482613im; 0.100415-0.0368004im 0.191058+2.21608e-18im 0.127569+0.00595898im 0.133935-0.000809274im; 0.0269368+0.0124297im 0.127569-0.00595898im 0.0696359+8.07705e-19im 0.160982+0.0119357im; 0.0618175-0.00482613im 0.133935+0.000809274im 0.160982-0.0119357im 0.538168+6.2422e-18im],0.850754
5,ComplexF64[0.0252676-5.15316e-19im 0.460293-0.201937im 0.0345184-0.112169im 0.0523977+0.0269789im; 0.460293+0.201937im 0.840408-5.55944e-18im 0.0389593-0.0829546im 0.398009-0.163884im; 0.0345184+0.112169im 0.0389593+0.0829546im 0.0335136+1.17097e-18im 0.0857868-0.0173382im; 0.0523977-0.0269789im 0.398009+0.163884im 0.0857868+0.0173382im 0.100811+4.90379e-18im],0.372044
6,ComplexF64[0.649931+1.87486e-17im 0.424936+0.00440416im 0.282991+0.103698im 0.191666-0.149527im; 0.424936-0.00440416im 0.234348-1.25509e-17im 0.0008755-0.017158im 0.213317-0.0304422im; 0.282991-0.103698im 0.0008755+0.017158im 0.0963281-5.15903e-18im 0.0588587+0.0317693im; 0.191666+0.149527im 0.213317+0.0304422im 0.0588587-0.0317693im 0.0193927-1.03861e-18im],0.419849
7,ComplexF64[0.14515-1.07165e-17im 0.034325+0.00329198im 0.0173378-0.147749im 0.337439-0.0562284im; 0.034325-0.00329198im 0.638826+1.13783e-17im 0.225689+0.0191218im 0.00496655+0.0218488im; 0.0173378+0.147749im 0.225689-0.0191218im 0.170675+2.3915e-18im 0.104672+0.0971487im; 0.337439+0.0562284im 0.00496655-0.0218488im 0.104672-0.0971487im 0.0453478-3.05336e-18im],0.413707
8,ComplexF64[0.14641+4.81778e-18im 0.25945+0.0312874im 0.0475538+0.179228im 0.179032+0.159568im; 0.25945-0.0312874im 0.38447-1.7881e-17im 0.00769586-0.0125402im 0.215844-0.0332038im; 0.0475538-0.179228im 0.00769586+0.0125402im 0.00583793-2.71512e-19im 0.0875601-0.0182768im; 0.179032-0.159568im 0.215844+0.0332038im 0.0875601+0.0182768im 0.463282+1.33348e-17im],0.481589
9,ComplexF64[0.0681384-7.54663e-19im 0.0817022-0.127218im 0.165434+0.00413061im 0.0920068+0.160421im; 0.0817022+0.127218im 0.0237851-1.00179e-18im 0.234899+0.05963im 0.0461291-0.0076163im; 0.165434-0.00413061im 0.234899-0.05963im 0.639026+4.73631e-18im 0.0844188+0.0242853im; 0.0920068-0.160421im 0.0461291+0.0076163im 0.0844188-0.0242853im 0.269051-2.97986e-18im],0.449166


In [5]:
using Printf
import Polyhedra
import LinearAlgebra

### Print floats with 4 decimal digits
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)
###


### Some useful objects
Pauli_matrices = [[0 1; 1 0], [0 -im; im 0], [1 0; 0 -1]]
Hadamard = [1 1; 1 -1]/sqrt(2)
phase_gate(phi) = [1 0; 0 exp(im*phi)]
###


### Useful functions for matrix manipulations
function index_to_array(k, dims) #= dims = [2, 2, 2] or something =#
    n_parties = length(dims)

    array = ones(n_parties)
    for i = 2:k
        array[n_parties] = array[n_parties] + 1
        for j in n_parties:-1:1
            if array[j] > dims[j]
                array[j-1] = array[j-1] + 1
                array[j] = 1
            end
        end
    end
    return array
end

function array_to_index(array, dims)
    n_parties = length(dims)
    index = 1
    for i = n_parties:-1:1
        prod = 1
        if i < n_parties
            for j = n_parties:(i+1)
                prod = prod*dims[j]
            end
        end
        index = index + (array[i] - 1)*prod
    end
    return Int64(index)
end

function partial_transpose(matrix, dims, axis) #= dims = [2, 2, 2] or something =#

    n_parties = length(dims)
    
    partially_transposed_matrix = copy(matrix)
    for i = 1:size(matrix)[1], j = 1:size(matrix)[2]
        
        array_i = index_to_array(i, dims)
        array_j = index_to_array(j, dims)
        
        new_array_i = copy(array_i)
        new_array_j = copy(array_j)
        
        new_array_i[axis] = array_j[axis]
        new_array_j[axis] = array_i[axis]

        new_index_i = array_to_index(new_array_i, dims)
        new_index_j = array_to_index(new_array_j, dims)

        
        partially_transposed_matrix[i, j] = matrix[new_index_i, new_index_j]
    end

    return partially_transposed_matrix
end

function partial_transpose(matrix::LinearAlgebra.Hermitian, dims, axis) #= dims = [2, 2, 2] or something =#    
    partially_transposed_matrix = copy(matrix)
    for i = 1:size(matrix)[1], j = 1:size(matrix)[2]
        
        array_i = index_to_array(i, dims)
        array_j = index_to_array(j, dims)
        
        new_array_i = copy(array_i)
        new_array_j = copy(array_j)
        
        new_array_i[axis] = array_j[axis]
        new_array_j[axis] = array_i[axis]

        new_index_i = array_to_index(new_array_i, dims)
        new_index_j = array_to_index(new_array_j, dims)

        
        partially_transposed_matrix.data[i, j] = matrix[new_index_i, new_index_j]
    end

    return partially_transposed_matrix
end

function partial_transpose(matrix::LinearAlgebra.Symmetric, dims, axis) #= dims = [2, 2, 2] or something =#

    n_parties = length(dims)
    
    partially_transposed_matrix = copy(matrix)
    for i = 1:size(matrix)[1], j = 1:size(matrix)[2]
        
        array_i = index_to_array(i, dims)
        array_j = index_to_array(j, dims)
        
        new_array_i = copy(array_i)
        new_array_j = copy(array_j)
        
        new_array_i[axis] = array_j[axis]
        new_array_j[axis] = array_i[axis]

        new_index_i = array_to_index(new_array_i, dims)
        new_index_j = array_to_index(new_array_j, dims)

        
        partially_transposed_matrix.data[i, j] = matrix[new_index_i, new_index_j]
    end

    return partially_transposed_matrix
end

function partial_trace(matrix, dims, axis)
    
    n_parties = length(dims)


    new_dims = ones(n_parties - 1)
    for i = 1:n_parties
        if i < axis
            new_dims[i] = dims[i]
        elseif i > axis
            new_dims[i-1] = dims[i]
        end
    end

    matrix_dimension = 1
    for i=1:(n_parties-1)
        matrix_dimension = Int64(matrix_dimension*new_dims[i])
    end

    
    new_matrix = zeros(typeof(matrix[1,1]), matrix_dimension,matrix_dimension)
    for i = 1:size(matrix)[1], j = 1:size(matrix)[2]
        array_i = index_to_array(i, dims)
        array_j = index_to_array(j, dims)

        if array_i[axis] == array_j[axis]
            
            new_array_i = ones(n_parties - 1)
            new_array_j = ones(n_parties - 1)

            for k = 1:n_parties
                if k < axis
                    new_array_i[k] = array_i[k]
                    new_array_j[k] = array_j[k]
                elseif k > axis
                    new_array_i[k-1] = array_i[k]
                    new_array_j[k-1] = array_j[k]
                end
            end

            new_index_i = array_to_index(new_array_i, new_dims)
            new_index_j = array_to_index(new_array_j, new_dims)

            new_matrix[new_index_i, new_index_j] = new_matrix[new_index_i, new_index_j] + matrix[i, j]
            
            
        end
    end

    return new_matrix
end

function swap_alice_bob(matrix; dims = [2,2])
    new_dims = dims[end:-1:1]
    new_matrix = copy(matrix)
    for i = 1:size(matrix)[1], j = 1:size(matrix)[2]
        new_array_i = index_to_array(i, dims)
        new_array_j = index_to_array(j, dims)

        old_array_i = new_array_i[end:-1:1]
        old_array_j = new_array_j[end:-1:1]

        old_index_i = array_to_index(old_array_i, dims)
        old_index_j = array_to_index(old_array_j, dims)

        new_matrix[i, j] = matrix[old_index_i, old_index_j]
    end
    return new_matrix
end
###


### Useful functions when dealing with qubits

#To specify a projective measurement for a qubit, it is only necessary to specify a bloch vector. This function transforms the bloch vector in the actual projector.
function vector_to_hermitian(measurement_vectors::AbstractMatrix)
    N = size(measurement_vectors)[1]
    vectors_dot_sigma = [sum(measurement_vectors[i,j]*Pauli_matrices[j] for j in 1:3) for i in 1:N]
    projectors = [(LinearAlgebra.I(2) + vectors_dot_sigma[i])/2 for i in 1:N]
    return projectors
end

function vector_to_hermitian(measurement_vectors::AbstractVector)
    N = length(measurement_vectors)
    vectors_dot_sigma = [sum(measurement_vectors[i][j]*Pauli_matrices[j] for j in 1:3) for i in 1:N]
    projectors = [(LinearAlgebra.I(2) + vectors_dot_sigma[i])/2 for i in 1:N]
    return projectors
end

#Transforms a bloch vector into its corresponding ket state (in the computational basis)
function bloch_vector_to_ket(vector)
    theta = acos(vector[3])
    phi = atan(vector[2], vector[1])
    return [cos(theta/2), exp(im*phi)*sin(theta/2)]
end

#To specify a dichotomic measurement, it is only necessary to specify one of the measurement operators. The following function constructs the whole measurements
function all_measurement_elements(finite_measurements)
    n_measurements = size(finite_measurements)[1]
    measurement_elements = [finite_measurements[1]]
	for i=2:2*n_measurements
		if iseven(i)
			push!(measurement_elements, LinearAlgebra.I(2)- measurement_elements[i-1])
		else
			push!(measurement_elements, finite_measurements[1+floor(Int64, i/2)])
		end
	end
    return measurement_elements
end

#Computes the unitary corresponding to a rotation in the Bloch sphere (see https://en.wikipedia.org/wiki/Euler_angles and https://qubit.guide/2.12-composition-of-rotations)
function rotation_to_unitary(rotation_matrix)
    phi = acos(rotation_matrix[3,3])
    if phi == 0
        alpha = atan(-rotation_matrix[1,2], rotation_matrix[1,1])
        beta = 0
    else
        alpha = atan(rotation_matrix[1, 3], -rotation_matrix[2, 3])
        beta = atan(rotation_matrix[3, 1], rotation_matrix[3, 2])
    end
    return phase_gate(alpha)*Hadamard*phase_gate(phi)*Hadamard*phase_gate(beta)
end

#Given state and measurements, constructs the associated behavior (in terms of correlators)
function quantum_to_correlators(input_state, measurements_Alice, measurements_Bob; proj = false, all_measurement_operators = false)
    if all_measurement_operators == true
        all_proj_Alice = measurements_Alice
        all_proj_Bob = measurements_Bob
    elseif proj == true
        all_proj_Alice = all_measurement_elements(measurements_Alice)
        all_proj_Bob = all_measurement_elements(measurements_Bob)
    else
        proj_Alice = vector_to_hermitian(measurements_Alice)
        all_proj_Alice = all_measurement_elements(proj_Alice)

        proj_Bob = vector_to_hermitian(measurements_Bob)
        all_proj_Bob = all_measurement_elements(proj_Bob)
    end

    n_projs_Alice = size(all_proj_Alice)[1]
    n_projs_Bob = size(all_proj_Bob)[1]

    observables_Alice = [all_proj_Alice[i] - all_proj_Alice[i+1] for i in 1:2:(n_projs_Alice-1)]
    observables_Bob = [all_proj_Bob[i] - all_proj_Bob[i+1] for i in 1:2:(n_projs_Bob-1)]

    n_observables_Alice = size(observables_Alice)[1]
    n_observables_Bob = size(observables_Bob)[1]


    all_observables = [kron(observables_Alice[i], LinearAlgebra.I(2)) for i in 1:n_observables_Alice]
    for j=1:n_observables_Bob
        push!(all_observables, kron(LinearAlgebra.I(2), observables_Bob[j]))
    end
    for i in 1:n_observables_Alice
        for j in 1:n_observables_Bob
            push!(all_observables, kron(observables_Alice[i],observables_Bob[j]))
        end
    end
    behavior = real.([LinearAlgebra.tr(all_observables[i]*input_state) for i in 1:size(all_observables)[1]])
    return behavior
end

#Given state and measurements, returns assemblage
function state_to_assemblage(input_state, measurements_Alice; proj = false, all_measurement_operators = false)
    if all_measurement_operators
        measurement_elements_Alice = measurements_Alice
    elseif proj
        measurement_elements_Alice = all_measurement_elements(measurements_Alice)
    else
        measurement_elements_Alice = all_measurement_elements(vector_to_hermitian(measurements_Alice))
    end
    assemblage = [partial_trace(kron(measurement_elements_Alice[i], LinearAlgebra.I(2))*input_state, [2, 2], 1) for i in 1:size(measurement_elements_Alice)[1]]
    return assemblage
end

#Writes a given state in its canonical form
function canonical_form(input_state; kill_Bobs_marginal = true)
    if kill_Bobs_marginal
        rho_B = partial_trace(input_state, [2, 2], 1)
        map = LinearAlgebra.sqrt(LinearAlgebra.inv(rho_B))
        input_state = kron(LinearAlgebra.I(2), map)*input_state*kron(LinearAlgebra.I(2), map)
        input_state = input_state/LinearAlgebra.tr(input_state)
    end
    T = real.([LinearAlgebra.tr(input_state*kron(Pauli_matrices[i], Pauli_matrices[j])) for i in 1:3, j in 1:3])
    if LinearAlgebra.Diagonal(T) == T
        return input_state
    else
        U, S, V = LinearAlgebra.svd(T)
        if LinearAlgebra.det(U) < 0
            U = -U
        end
        if LinearAlgebra.det(V) < 0
            V = -V
        end
        U_A = rotation_to_unitary(U')
        U_B = rotation_to_unitary(V')

        return kron(U_A, U_B)*input_state*kron(U_A', U_B')
    end
end

#Generates a random state (uniformly sampled according to the Hilber-Schimidt Metric)
function random_state(dim)
	G = randn(ComplexF64, (dim, dim))
	return G*G'/LinearAlgebra.tr(G*G')
end

#Generates a random unit vector
function random_unit_vectors(n; dims = 3)
	vector = [randn(dims) for i in 1:n]
	normalized_vectors = [vector[i]/LinearAlgebra.norm(vector[i]) for i in 1:n]
	return normalized_vectors
end

function random_pure_state(dim)
    unnormalized_pure_ket = randn(ComplexF64, dim)
    random_pure_ket = unnormalized_pure_ket/(LinearAlgebra.norm(unnormalized_pure_ket))
    random_pure_state = random_pure_ket*random_pure_ket'
    return random_pure_state
end

#Tells whether a two-qubit quantum state is separable
function is_separable(input_state)
    return real(LinearAlgebra.det(partial_transpose(input_state, [2, 2], 2))) >= 0
end
###



### Useful functions for Bell scenarios
#Enumerates the deterministic behaviors (in the p(a|x) description) of one party (useful for the old method)
function single_party_deterministic_probs(n_measurements)
	behaviors = zeros((2^n_measurements, 2*n_measurements))
	for i = 1:2^n_measurements
		binary = bitstring(i)[end-n_measurements+1:end]
		for j = 1:n_measurements
			behaviors[i, (2*j-1) + parse(Int, binary[j])] = 1
		end
	end
	return behaviors
end

#Enumerates the deterministic behaviors (in the correlators description) of one party (useful for the old method)
function single_party_deterministic_correlators(n_measurements)
    behaviors = zeros((2^n_measurements, n_measurements))
    for i = 1:2^n_measurements
        binary = bitstring(i)[end-n_measurements+1:end]
            for j = 1:n_measurements
                behaviors[i, j] = (-1)^parse(Int, binary[j])
            end
    end
    return behaviors
end

#Enumerates the deterministic behaviors of a bipartite Bell scenario (in terms of correlators)
function deterministic_vertices(n_measurements_Alice, n_measurements_Bob)
    n_vertices = 2^(n_measurements_Alice + n_measurements_Bob)
    behavior_length = n_measurements_Alice + n_measurements_Bob + n_measurements_Alice*n_measurements_Bob
    local_vertices = zeros((n_vertices, behavior_length))
    for i = 1:n_vertices
        binary = bitstring(i-1)[end-(n_measurements_Alice + n_measurements_Bob-1):end]
        local_vertices[i, 1:n_measurements_Alice] .= [(-1)^parse(Int64, binary[j]) for j in 1:n_measurements_Alice]
        local_vertices[i, n_measurements_Alice+1:n_measurements_Alice+n_measurements_Bob] .= [(-1)^parse(Int64, binary[j]) for j in n_measurements_Alice+1:n_measurements_Bob+n_measurements_Alice]
        for j in 1:n_measurements_Alice*n_measurements_Bob
            x = 1 + floor(Int64, (j-1)/n_measurements_Bob)
            y = 1 + (j-1)%n_measurements_Bob
            local_vertices[i, n_measurements_Alice + n_measurements_Bob + j] = local_vertices[i, x]*local_vertices[i, n_measurements_Alice + y]
        end
    end
    return local_vertices
end

#Defines a matrix A such that Ab <= ones for all NS behaviors (in terms of correlators)
# - a*A_x - b*B_y - a*b*A_x B_y <= 1
function NSmatrix(n_measurements_Alice, n_measurements_Bob)
    behavior_size = n_measurements_Alice + n_measurements_Bob + n_measurements_Alice*n_measurements_Bob
    n_inequalities = (2^2)*n_measurements_Alice*n_measurements_Bob
    A = zeros((n_inequalities, behavior_size))
    for i = 1:n_measurements_Alice*n_measurements_Bob
        x = floor(Int64, 1 + (i-1)/n_measurements_Bob)
        y = 1+(i-1)%n_measurements_Bob
        for j = 1:4
            binary = bitstring(j)[end-1:end]
            A[4*(i-1)+j, x] = -(-1)^parse(Int64, binary[1])
            A[4*(i-1)+j, n_measurements_Alice + y] = -(-1)^parse(Int64, binary[2])
            A[4*(i-1)+j, n_measurements_Alice + n_measurements_Bob + i] = -(-1)^parse(Int64, binary[1])*(-1)^parse(Int64, binary[2])
        end
    end
    return A
end
###









### Useful functions for manipulating polytopes

#For a given set of vertices describing a polytope, this function computes the polytope description in terms of inequalities
function vertices_to_facets(vertices)
    half_space_rep = Polyhedra.MixedMatHRep(Polyhedra.doubledescription(Polyhedra.vrep(vertices)))
    facet3D_vectors = [half_space_rep.A[i, 1:end] for i in 1:size(half_space_rep.A)[1]]
    offsets = half_space_rep.b
    return facet3D_vectors, offsets
end

#For a given set of facets describing a polytope, this function computes the maximum radius of a inner sphere
function shrinking_factor(facet3D_vectors, offsets)
    radius = minimum([abs(offsets[i])/LinearAlgebra.norm(facet3D_vectors[i]) for i in eachindex(offsets)])
    return radius
end

#For a given set of vertices defining a polytope, this function computes the maximum radius of a inner sphere
function shrinking_factor(vertices)
    facet3D_vectors, offsets = vertices_to_facets(vertices)
    radius = minimum([abs(offsets[i])/LinearAlgebra.norm(facet3D_vectors[i]) for i in eachindex(offsets)])
    return radius
end

#This function computes all the combinations with N elements of the set of points
function all_tuples(points, N)
    n_points = size(points)[1]
    tuples = []
    for i=1:2^n_points
        binary = bitstring(i)[end - n_points + 1: end]
        if sum(parse(Int64, binary[j]) for j in eachindex(binary)) == N
            new_tuple = []
            for k in 1:n_points
                if binary[k] == '1'
                    push!(new_tuple, points[k])
                end
            end
            push!(tuples, new_tuple)
        end
    end
    return tuples
end

#A nice family of polytopes
function fibonacci_polytope(n_vertices)
    points = Vector{Vector{Float64}}()
    phi = pi*(sqrt(5) - 1) #golden ration in radians
    for i in 1:n_vertices
        z = -1 + ((i-1)/(n_vertices-1))*2
        radius = sqrt(1 - z^2)

        theta = phi*i
        x = radius*cos(theta)
        y = radius*sin(theta)
        push!(points, [x, y, z])
    end
    return points
end
###




fibonacci_polytope (generic function with 1 method)

In [7]:

file_polytope72cov = parse.(Float64, readlines("/content/polytope72cov.txt")[1:end])
polytope72cov = [[file_polytope72cov[i], file_polytope72cov[i+1], file_polytope72cov[i+2]] for i in 1:3:216]
outer_polytope72cov = polytope72cov/shrinking_factor(polytope72cov)


72-element Vector{Vector{Float64}}:
 [0.7030, 0.6719, 0.3571]
 [0.5446, 0.8813, 0.0000]
 [0.0000, 0.5446, -0.8813]
 [-0.5446, 0.8813, 0.0000]
 [0.0000, 0.5446, 0.8813]
 [0.8813, -0.0000, 0.5446]
 [-0.8813, -0.0000, 0.5446]
 [-0.0000, -0.5446, 0.8813]
 [-0.8813, -0.0000, -0.5446]
 [-0.5446, -0.8813, 0.0000]
 ⋮
 [0.7944, 0.4328, -0.5049]
 [-0.9394, 0.3431, -0.2703]
 [0.3431, -0.2703, -0.9394]
 [0.7030, -0.6719, -0.3571]
 [-0.7030, -0.6719, 0.3571]
 [-0.2703, -0.9394, 0.3431]
 [0.2703, 0.9394, 0.3431]
 [-0.1450, -0.1478, -1.0151]
 [0.8480, -0.5823, 0.1224]

In [8]:
using JuMP
using Mosek
using MosekTools
using LinearAlgebra

# your code here



### Preliminary functions for Chau's method

#Given three points in 3d, returns the plane that passes through them
function three_points_to_plane(points)
    a_vec = points[2] - points[1]
    b_vec = points[3] - points[1]
    normal_vector = LinearAlgebra.cross(a_vec, b_vec)
    offset = LinearAlgebra.dot(points[1], normal_vector)
    return normal_vector, offset
end

#Given a finite set of points, returns all triples composed of such points
function all_triples(points)
    n_points = size(points)[1]
    triples = []
    for i=1:(n_points-2)
        for j in (i+1):(n_points-1)
            for k in (j+1):n_points
                push!(triples, [points[i], points[j], points[k]])
            end
        end
    end
    return triples
end

#Given a finite set of points in 3d, all_planes gets all the planes that pass through at least three of those points
function all_planes(polytope_vertices)
    triples = all_triples(polytope_vertices)
    normal_vectors = []
    offsets = []
    for i = 1:size(triples)[1]
        new_vec, new_offset = three_points_to_plane(triples[i]) 
        push!(normal_vectors, new_vec)
        push!(offsets, new_offset)
    end
    return normal_vectors, offsets
end

#Chau's method
function critical_radius(input_state, polytope_vertices = polytope72cov)
    normal_vectors, offsets = all_planes(polytope_vertices)
    n_vertices = size(polytope_vertices)[1]
    n_normal_vectors = length(offsets)

    canonical_state = canonical_form(input_state; kill_Bobs_marginal = true)

    a = real.([LinearAlgebra.tr(Pauli_matrices[i]*partial_trace(canonical_state, [2, 2], 2)) for i in 1:3])
    T = real.([LinearAlgebra.tr(canonical_state*kron(Pauli_matrices[i], Pauli_matrices[j])) for i in 1:3, j in 1:3])

    model = Model(Mosek.Optimizer)
    set_silent(model)

    r = @variable(model)
    @variable(model, probs[1:n_vertices] .>= 0)

    upper_bound_radius = [@expression(model, sum(probs[j]*abs(-offsets[i] + normal_vectors[i]'*polytope_vertices[j])/LinearAlgebra.norm(-offsets[i]*a + T*normal_vectors[i]) for j in 1:n_vertices)) for i in 1:n_normal_vectors]

    for i=1:n_normal_vectors
        @constraint(model, r <= upper_bound_radius[i])
    end

    @constraint(model, sum(probs) == 1)
    @constraint(model, sum(probs[j]*polytope_vertices[j] for j in 1:n_vertices) .== 0)

    @objective(model, Max, r)

    optimize!(model)

    return objective_value(model)
end


critical_radius (generic function with 2 methods)

In [9]:
using Parquet, DataFrames, PyCall, Dates

# Initialize the CriticalRadius column with NaN values
julia_df[!, :CriticalRadius] = NaN * ones(size(julia_df, 1))

data_to_process = julia_df[11111:end, :]

batch_size = 3000
total_rows = size(data_to_process, 1)
num_batches = ceil(Int, total_rows / batch_size)

s_factor=shrinking_factor(polytope72cov)

function process_batch(start_idx, end_idx, data_to_process, batch)    
    tasks = []
    start_time = Dates.now()  # Start time for the batch

    for i in start_idx:end_idx
        task = Threads.@spawn begin
            try
                # Assuming `critical_radius` takes the state and polytope vertices, and `ComplexArray` is the required input state.
                data_to_process[i, :CriticalRadius] = critical_radius(data_to_process[i, :State])
            catch e
                println("Error processing row $i: ", e)
                data_to_process[i, :CriticalRadius] = NaN
            end
        end
        push!(tasks, task)
    end

    # Wait for all tasks to complete
    for task in tasks
        fetch(task)
    end

    end_time = Dates.now()  # End time for the batch
    elapsed_time = end_time - start_time
    print("Batch $batch took $(Dates.value(elapsed_time)/1000) seconds to complete.\n")

    # Save the current batch to a pickle file
    subset_df = data_to_process[start_idx:end_idx, :]

    # Extract columns from the Julia DataFrame
    complex_array_col = subset_df.State
    inner_radius_col = subset_df.CriticalRadius

    # Calculate the shrunk radius
    outer_radius_col = inner_radius_col ./ s_factor

    # Create a dictionary to pass to pandas
    data_dict = Dict(
        "State" => collect(complex_array_col),
        "Inner Radius" => collect(critical_radius_col)
        "Outer Radius" => collect(outer_radius_col)
    )

    # Import pandas through PyCall
    pd = pyimport("pandas")

    # Convert the dictionary to a pandas DataFrame
    pd_df = pd.DataFrame(data_dict)

    # Save the DataFrame to a pickle file
    pickle_filename = "ML_dataset_batch_$batch.pkl"
    pd_df.to_pickle(pickle_filename)

    # Print the head of the current batch DataFrame
    print("Processed and saved batch $batch:\n")
    println(pd_df.head())

    # Clear memory used by pandas DataFrame
    pd_df = nothing
    data_dict = nothing
    complex_array_col = nothing
    boolean_value_col = nothing
    critical_radius_col = nothing

    # Call garbage collector
    GC.gc()
end

for batch in 1:num_batches
    print("Starting to process $batch")
    # Determine the range for the current batch
    start_idx = (batch - 1) * batch_size + 1
    end_idx = min(batch * batch_size, total_rows)

    # Process the batch
    process_batch(start_idx, end_idx, data_to_process, batch)
end


Starting to process 1Error processing row 2027: ErrorException("Invalid coefficient -Inf on variable probs[1].")
Error processing row 799: ErrorException("Invalid coefficient -Inf on variable probs[1].")
Batch 1 took 29712.3080 seconds to complete.
Processed and saved batch 1:
PyObject    Inner Radius                                              State  SDP Value
0      0.095315  [[(0.25946882-2.23980269e-17j), (0.15164606-0....   0.365470
1      0.748250  [[(0.07600106-1.76968741e-18j), (0.62059151-0....   0.342594
2      0.404288  [[(0.1343543-1.23758325e-17j), (0.0872655-0.00...   0.462462
3      0.522863  [[(0.35989324-1.3683483e-17j), (0.06017235-0.0...   0.519031
4      0.293348  [[(0.40600461-1.01701866e-17j), (0.23657849+0....   0.209035
Starting to process 2Error processing row 5816: ErrorException("Invalid coefficient -Inf on variable probs[1].")
Error processing row 3079: ErrorException("Invalid coefficient -Inf on variable probs[1].")
Batch 2 took 28887.9970 seconds to compl